In [4]:
import pandas as pd
import numpy as np
import io
from tqdm import tqdm
from itertools import islice
import langid
import re
from concurrent import futures
import swifter
import asyncio
import nltk
# import vaex
from nltk import  pos_tag
from nltk import WordNetLemmatizer
import modin.pandas as pdmd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import multiprocessing as mp


import warnings
warnings.filterwarnings("ignore")

# Создание масенького датасета из большого

Нашла я датасет гитхабовских проблем на 2гб

In [ ]:
data = pd.read_csv("github_issues.csv")
data

,issue_url,issue_title,body
0,"""https://github.com/zhangyuanwei/node-images/i...",can't load the addon. issue to: https://github...,can't load the addon. issue to: https://github...
1,"""https://github.com/Microsoft/pxt/issues/2543""",hcl accessibility a11yblocking a11ymas mas4.2....,user experience: user who depends on screen re...
2,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1265: issue 1264: issue 1261: issue 1260...,┆attachments: <a href= https:& x2f;& x2f;githu...
3,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1266: issue 1263: issue 1262: issue 1259...,gitlo = github x trello\n---\nthis board is no...
4,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1288: issue 1285: issue 1284: issue 1281...,┆attachments: <a href= https:& x2f;& x2f;githu...
...,...,...,...
5332148,"""https://github.com/bayborodin/ror-full-3/issu...","создать модуль instancecounter, содержащий сле...","методы класса: - instances, который возвращает..."
5332149,"""https://github.com/eclipse/paho.mqtt.java/iss...",at org.eclipse.paho.client.mqttv3.internal.cli...,- bug exists release version 1.1.0 master bran...
5332150,"""https://github.com/rzwitserloot/lombok/issues...",java.lang.linkageerror: loader constraint viol...,java.lang.linkageerror: loader constraint viol...
5332151,"""https://github.com/Gizra/productivity/issues/...",node : pdoexception: sqlstate 40001 : serializ...,view details in rollbar: https://rollbar.com/b...


In [ ]:
data.describe()

,issue_url,issue_title,body
count,5332153,5332153,5332153
unique,5315477,4862563,5041808
top,"""https://github.com/cmty-test/cmty-repository-...",first from flow in uk south,first from flow in uk south
freq,297,90133,90133


In [ ]:
slice_data = data.sample(frac=1, random_state=42)[:500_000]
slice_data = slice_data.reset_index(drop=True)

In [ ]:
slice_data.to_csv("github_issues_slice.csv", index=False)

## Проведу мини-анализ по ускорению обработки данных

Для начала мне нужно выделить из исходного датасета только текст на английском языке (мне так хочется). Для этой задачи уже можно попробовать ускорять

In [2]:
def is_en(text):
    return text if langid.classify(text)[0] == 'en' else pd.NA

In [3]:
def is_en_md(text):
    return text if langid.classify(text)[0] == 'en' else pdmd.NA

In [4]:
def is_en_bool(text):
    return langid.classify(text)[0] == 'en'

### read_csv

In [5]:
%%timeit

slice_data = pd.read_csv("github_issues_slice.csv")[:1000]

7.42 s ± 526 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit

slice_data_md = pdmd.read_csv("github_issues_slice.csv")[:1000]

3.71 s ± 307 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Apply by pandas and map

In [15]:
%%timeit

to_save = slice_data[slice_data.body.map(is_en_bool)]
# slice_data = slice_data[to_save.index]
len(to_save)

6.15 s ± 215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit

res = slice_data.body.apply(is_en)
res.dropna()

7.18 s ± 413 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

res = slice_data_md.body.apply(is_en_md)
res.dropna()

7.18 s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### future ThreadPoolExecutor

In [16]:
%%timeit

with futures.ThreadPoolExecutor() as executor:
    res = pd.Series(executor.map(is_en, slice_data.body))
    res.dropna()

3.55 s ± 188 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit

with futures.ThreadPoolExecutor() as executor:
    res = pdmd.Series(executor.map(is_en_md, slice_data_md.body))
    res.dropna()

4.24 s ± 376 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### future ProcessPoolExecutor

In [ ]:
%%writefile ProcessPoolExecutor_pd.py

import pandas as pd
import langid
import time
from concurrent import futures
import numpy as np

def is_en(text):
    return langid.classify(text)[0] == 'en'

def process_series(series: pd.Series):
    return series.map(is_en)

def main(df):
    n = 4
    parts = np.array_split(df.body, n)

    print("START TIME")
    start = time.perf_counter()

    with futures.ProcessPoolExecutor() as executor:
        res = executor.map(process_series, parts)

    print(f"END TIME: {time.perf_counter() - start}")
    print(res)


if __name__ == '__main__':
    df = pd.read_csv("github_issues_slice.csv")[:1000]

    main(df)

Overwriting ProcessPoolExecutor_pd.py


In [ ]:
!python ProcessPoolExecutor_pd.py

START TIME
END TIME: 10.9749333
<generator object _chain_from_iterable_of_lists at 0x000001E022B25740>


In [ ]:
%%writefile ProcessPoolExecutor_pdmd.py

import pandas as pd
import modin.pandas as pdmd
import langid
import time
from concurrent import futures


def is_en(text):
    return text if langid.classify(text)[0] == 'en' else pd.NA

def main(df):
    print("START TIME")
    start = time.perf_counter()

    with futures.ProcessPoolExecutor() as executor:
        res = pdmd.Series(executor.map(is_en, df.body))

    print(len(res.dropna()))
    print(f"END TIME: {time.perf_counter() - start}")

if __name__ == '__main__':
    df = pdmd.read_csv("github_issues_slice.csv")[:1000]

    main(df)

Writing ProcessPoolExecutor_pdmd.py


In [ ]:
!python ProcessPoolExecutor_pdmd.py

### swifter

In [ ]:
%%timeit

res = slice_data.body.swifter.apply(is_en)
res.dropna()

Pandas Apply: 100%|██████████| 1000/1000 [00:08<00:00, 111.75it/s]

8.88 s ± 885 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Process

In [ ]:
%%writefile process_mp.py

import pandas as pd
import langid
import time
from concurrent import futures
import numpy as np
import multiprocessing as mp


def is_en_bool(text):
    return langid.classify(text)[0] == 'en'

def process_series(series: pd.Series):
    return series.map(is_en_bool)

def main_n_process(parts):
    processes = [
        mp.Process(target=process_series, args=(part,))
        for part in parts
    ]
    
    for process in processes:
        process.start()

    print("START TIME")
    start = time.perf_counter()
    
    for process in processes:
        process.join()

    print(f"END TIME: {time.perf_counter() - start}")
    return process

if __name__ == '__main__':
    slice_data = pd.read_csv("github_issues_slice.csv")[:1000]
    n = 4
    parts = np.array_split(slice_data.body, n)
    result = main_n_process(parts)
    print(result)

Overwriting process_mp.py


In [ ]:
!python process_mp.py

START TIME
END TIME: 7.654426100000002
<Process name='Process-4' pid=10088 parent=13892 stopped exitcode=0>


### vaex

In [ ]:
dv = vaex.from_csv("github_issues_slice.csv", convert=True, chunk_size=1_000_000)

In [ ]:
dv = vaex.open('github_issues_slice.csv.hdf5')[:1000]

In [ ]:
def is_en(text):
    return np.where(langid.classify(text)[0] == 'en', text, np.nan)
    # return text if langid.classify(text)[0] == 'en' else np.nan

In [ ]:
dv.body = dv.body.apply(is_en)

In [ ]:
%%time
len(dv.dropna())

Wall time: 27 s


1000

# Result

In [73]:
def is_en(text):
    return text if langid.classify(text)[0] == 'en' else pd.NA

In [74]:
data = pd.read_csv("github_issues_slice.csv")[:10_000]

In [75]:
with futures.ThreadPoolExecutor() as executor:
    data.body = pd.Series(executor.map(is_en, data.body))

In [76]:
len(data[data.body.isnull()])

2298

In [77]:
data = data.dropna()

In [78]:
data

,issue_url,issue_title,body
0,"""https://github.com/atais/angular-eonasdan-dat...",manually entered dates issues,i use format 'yyyy-mm-dd' option and bind ng-m...
1,"""https://github.com/conveyal/analysis-ui/issue...",highlight segment on map when editing speed,when editing the speed of a segment i.e. when ...
2,"""https://github.com/Tat5ato/Phantasmic-Mind/is...",concept art for the otherworld,"in general, the otherworld is craggy and organ..."
3,"""https://github.com/qmlweb/qmlweb/issues/420""",mousearea and touch event in mobile browser,"hello, in the master branch, mousearea doesn't..."
4,"""https://github.com/pybel/pybel/issues/174""","function to drop graph store and edge store, b...",this function should go in the cache manager. ...
...,...,...,...
34994,"""https://github.com/KhronosGroup/glTF-Blender-...",implement current and outstanding gltf 2.0 ext...,"gltf 2.0 will have several new extensions, whi..."
34996,"""https://github.com/PX4/Firmware/issues/6555""","local_position_ned.z not stable, varying errat...",we are trying to do off-board control for pixh...
34997,"""https://github.com/zertosh/invariant/issues/29""",support es modules,invariant only provides a commonjs interface e...
34998,"""https://github.com/atom/autocomplete-emojis/i...",image links broken in readme,just letting you know that the image links use...


In [79]:
data = data.reset_index(drop=True)

# Обработка данных

In [12]:
data.body[0]

"i use format 'yyyy-mm-dd' option and bind ng-model= formdata.date to my field. now if i enter '2015-03-05aaaaa', the date is displayed as '2015-03-05', but my $scope.formdata.date becomes '2015-03-05aaaaa'. so even if i write error message to user that date is invalid, in input field it looks like totally valid value '2015-03-05'. how do i stop datetimepicker from automatically correcting my dates? this misleads users."

In [13]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Sentence segmentation

In [14]:
def sentence_segmentation(text: str) -> list:
    if re.match(r'[\.!\?;]', text[-1]):
        text = text[:-1]
    return re.split(r'[\.!\?;]\s', text.lower())

## Tokenization

In [15]:
def tokenization(sentences: list) -> list:
    return [re.split(r'[,:(\s\-)]*\s', s) for s in sentences]

## Lemmatization

In [1]:
from nltk.corpus import wordnet as wn

In [2]:
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wn.ADJ,
        'V': wn.VERB,
        'N': wn.NOUN,
        'R': wn.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wn.NOUN

In [4]:
def lemmatization(sentences: list) -> list:
    sentences_tag  = [pos_tag(s) for s in sentences] # получаем теги слов каждого предложения

    lemmatizer = WordNetLemmatizer()
    lemm_sentences = []
    for sent in sentences_tag:
        pos_tagged = [(word, get_wordnet_pos(tag)) for word, tag in sent]
        lemm_sentences.append([lemmatizer.lemmatize(word, tag) for word, tag in pos_tagged])

    return lemm_sentences

## Delete stopwords

In [19]:
stop_words = set(stopwords.words('english')).union({'', ' '})

In [20]:
def del_stopwords(sentences: list) -> list:
    upd_sentences = []
    re_sub = lambda x: re.sub(r"[\+=\t\r\n,;:\*'\"]+","", x)
    union_sentences = lambda x: list(set().union(*x))

    for sent in sentences:
        upd_sentences.append([
            re_sub(word) for word in sent if re_sub(word) not in stop_words and len(word) not in [1, 2]
        ])
    
    return union_sentences(upd_sentences)

## All

In [21]:
def preprocessing_text(text: str) -> list:
    return del_stopwords(lemmatization(tokenization(sentence_segmentation(text))))

In [80]:
with futures.ThreadPoolExecutor() as executor:
    data['words_body'] = pd.Series(executor.map(preprocessing_text, data.body))

In [81]:
with futures.ThreadPoolExecutor() as executor:
    data['words_title'] = pd.Series(executor.map(preprocessing_text, data.issue_title))

## Vectorization

In [24]:
def load_vectors(fname, limit):
  fin = io.open(fname, 'r', encoding = 'utf-8', newline = '\n', errors = 'ignore')
  n, d = map(int, fin.readline().split())
  data = {}
  for line in tqdm(islice(fin, limit), total = limit):
    tokens = line.rstrip().split(' ')
    data[tokens[0]] = np.array(list(map(float, tokens[1:])))
  return data

vecs = load_vectors('crawl-300d-2M.vec', 1_000_000)  

100%|██████████| 1000000/1000000 [03:32<00:00, 4699.63it/s]


In [25]:
def dummy_fun(doc):
    return doc

In [82]:
tfidf_body = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None)
tfidf_body.fit(data.words_body)

tfidf_title = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None)
tfidf_title.fit(data.words_title)

TfidfVectorizer(preprocessor=<function dummy_fun at 0x00000238257A5820>,
                token_pattern=None,
                tokenizer=<function dummy_fun at 0x00000238257A5820>)

In [27]:
def vectorization_body(words: list):
    return tfidf_body.transform([words]).toarray().squeeze()

def vectorization_title(words: list):
    return tfidf_title.transform([words]).toarray().squeeze()

In [83]:
with futures.ThreadPoolExecutor() as executor:
    data['vectors_body'] = pd.Series(executor.map(vectorization_body, data.words_body))

In [ ]:
with futures.ThreadPoolExecutor() as executor:
    data['vectors_title'] = pd.Series(executor.map(vectorization_title, data.words_title))

In [30]:
dim = 300
zero = sum(vecs.values()) / len(vecs)

In [ ]:
vocab_body = np.zeros((len(tfidf_body.vocabulary_.keys()), dim))
for key in tqdm(tfidf_body.vocabulary_.keys()):
  vocab_body[tfidf_body.vocabulary_[key]] = vecs.get(key, zero)

100%|██████████| 82847/82847 [00:05<00:00, 14993.70it/s]


In [ ]:
vocab_title = np.zeros((len(tfidf_title.vocabulary_.keys()), dim))
for key in tqdm(tfidf_title.vocabulary_.keys()):
  vocab_title[tfidf_title.vocabulary_[key]] = vecs.get(key, zero)

100%|██████████| 16719/16719 [00:00<00:00, 45359.94it/s]


In [33]:
data['vectors_body'] = np.array(data.vectors_body.tolist()).dot(vocab_body)
data['vectors_title'] = np.array(data.vectors_title.tolist()).dot(vocab_title)

In [34]:
# data.to_pickle("data_temp")